In [2]:
import os
import logging
import pandas as pd
import getpass
from datetime import datetime
logger = logging.getLogger('LabJournal')
logging.basicConfig(level=logging.DEBUG)

In [3]:
import sys
sys.path.append("../../../") # need this if labjournal is not installed

from simdb.utils.fileFinder import find_files
from simdb.databaseModel import *
from simdb.databaseAPI import *
import simdb.utils.tpr_parser as tpr_parser


In [ ]:
SIMS = '/home/andrejb/Research/SIMS'
GMXBIN="/home/soft/gromacs/gromacs-2018/inst/shared/bin/"
os.environ['GMXBIN'] = GMXBIN

user = getpass.getuser()

In [ ]:
logger.info('create_database:create_database: FIND FILES')
ERRORS=False
WARNINGS=False

In [ ]:
%%time
SIM_IDS=[]
PATHS=[]
METAS=[]

for fname in find_files(pattern = 'topol.tpr', path = SIMS, dir_ignore = ['data']):
    path =  os.path.dirname(fname)
    sim_id = os.path.basename(path)
    try:
        meta = pd.Series.from_csv(os.path.join(path, "meta.csv"))
    except IOError:
        meta = pd.Series()
    SIM_IDS.append(sim_id)
    PATHS.append(path)
    METAS.append(meta)

In [ ]:
db = 'andrej_raw.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)

In [ ]:
%%time
for i, entry_id in enumerate(SIM_IDS):
    path = PATHS[i]
    tprfile = os.path.join(path,'topol.tpr')
    
    mapped_keywords = tpr_parser.main(tprfile)
    
    meta = METAS[i]
    try:
        description = meta['note']
        del meta['note']
    except KeyError:
        description = ""
    
    if 'created_on' in meta:
        created_on  = meta['created_on']
        del meta['created_on']
    else:
        created_on = datetime.fromtimestamp(os.path.getmtime(tprfile))
        
    sim = store_dict(
        entry_id    = entry_id,
        path        = path,
        sim_type    = "GROMACS",
        description = description,
        created_on  = created_on,
        owner       = user,
        raw_mdp_parameters = mapped_keywords,
        raw_keywords = meta,
    )
    
    session.add(sim)
    session.flush()

session.commit()
session.close()

In [ ]:
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.query(Main).all()

In [ ]:
session.close()

## Add groups to DB

In [23]:
db = 'andrej_raw.db'

engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
s = Session()
# setup_database(engine)

In [24]:
e = s.query(Main).filter(Main.entry_id == "2017_06_09_GfM_SMmin_2ub_k48_01").one()

In [25]:
e.groups

[Groups(name='K48 linked')]

In [26]:
s.query(Groups).all()

[Groups(name='K48 linked')]

In [33]:
s.close()

In [18]:
db_path = "andrej_raw.db"
table = getEntryTable(db_path)

In [19]:
sel = table[selectByKeyword(table, "linker", "k48")]

In [65]:
def add_to_group(db_path, entry_id, groupname):
    """Add all simulations in entry_id to group.
    
    Args:
        db_path: string, path to database
        entry_id: list, entry IDs of simulations which 
                  should be added to group
        groupname: string, name of group
    """
    
    # open databae
    s = openDatabase(db_path)
    
    # get group if already in DB or create new group
    try:
        group = s.query(Groups).filter(Groups.name == groupname).one()
    except NoResultFound:
        group = Groups(name=groupname)
    
    entries = s.query(Main).filter(Main.entry_id.in_(entry_id)).all()
    
    # add
    for entry in entries:
        group.entries.append(entry)
    
    # commit and close
    s.commit()
    s.close()

In [62]:
def get_entry_group_table(db_path, groupname, columns=None):
    """Get pandas table of all entries in certain group.
    
    Args:
        db_path: string, path to database
        groupname: string, name of group
        columns: list, columns which should be displayed
    """
    
    # open databae
    s = openDatabase(db_path)
    
    group = s.query(Groups).filter(Groups.name == groupname).one()
    q = s.query(Main).filter(Main.groups.any(id = group.id))
    
    # get entries as pandas table
    df = pd.read_sql(q.statement, s.bind, index_col="id")
    
    s.close()
    
    # convert output
    if columns is not None:
        df = df[columns]
    
    return df

In [22]:
addGroup(db_path, sel.entry_id, "K48 linked")

In [63]:
getEntryGroup(db_path, "K48 linked", columns=["entry_id", "path"])

,entry_id,path
id,,
4,2017_06_09_GfM_SMmin_2ub_k48_01,/home/andrejb/Research/SIMS/2017_06_09_GfM_SMm...
17,2017_08_01_2ub_k48_01_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...
27,2017_09_26_GfM_SMmin_rnd_k48_39,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...
28,2017_06_28_GfM_SMmin_rnd_k48_7,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...
39,2017_09_26_GfM_SMmin_rnd_k48_40,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...
42,2017_06_28_GfM_SMmin_rnd_k48_22,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...
61,2017_09_26_GfM_SMmin_rnd_k48_35,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...
65,2017_08_01_2ub_k48_02_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...
71,2017_06_28_GfM_SMmin_rnd_k48_19,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...


In [48]:
group = s.query(Groups).filter(Groups.name == "K48 linked").one()

In [49]:
q = s.query(Main).filter(Main.groups.any(id = group.id))

In [50]:
q.statement

<sqlalchemy.sql.annotation.AnnotatedSelect at 0x7f4678c238d0; AnnotatedSelect object>

In [59]:
pd.read_sql(q.statement, s.bind, index_col="id")[["entry_id", "path", "description"]]

,entry_id,path,description
id,,,
4,2017_06_09_GfM_SMmin_2ub_k48_01,/home/andrejb/Research/SIMS/2017_06_09_GfM_SMm...,Simulations of atomistic dimers. Start from mi...
17,2017_08_01_2ub_k48_01_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,
27,2017_09_26_GfM_SMmin_rnd_k48_39,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,Simulations of atomistic dimers. Start from ra...
28,2017_06_28_GfM_SMmin_rnd_k48_7,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,Simulations of atomistic dimers. Start from ra...
39,2017_09_26_GfM_SMmin_rnd_k48_40,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,Simulations of atomistic dimers. Start from ra...
42,2017_06_28_GfM_SMmin_rnd_k48_22,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,Simulations of atomistic dimers. Start from ra...
61,2017_09_26_GfM_SMmin_rnd_k48_35,/home/andrejb/Research/SIMS/2017_09_26_GfM_SMm...,Simulations of atomistic dimers. Start from ra...
65,2017_08_01_2ub_k48_02_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,
71,2017_06_28_GfM_SMmin_rnd_k48_19,/home/andrejb/Research/SIMS/2017_06_28_GfM_SMm...,Simulations of atomistic dimers. Start from ra...


## Setup an empty DB

In [ ]:
db = 'empty.db'
# remove old DB
try:
    os.remove(db)
    logger.info('create_database:create_database: removed old file: %s', db)
except:
    pass
engine = create_engine('sqlite:///{}'.format(db) , echo=False) #  if we want spam

# Establishing a session
Session = sessionmaker(bind=engine)
session = Session()
setup_database(engine)
session.close()